## Import Library

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate

In [2]:
data_path_open_source = "/home/neemias/PerceptSent-LLM-approach/data/"
data_path_open_ai = "/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/"
data_paths = [os.path.join(data_path_open_source, f) 
              for f in os.listdir(data_path_open_source) if f.endswith('.csv')] + [os.path.join(data_path_open_ai, f)
                                                              for f in os.listdir(data_path_open_ai) if f.endswith('.csv')]
del data_paths[6]
data_paths[:10]

['/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p3.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha3_p2neg.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha4_p5.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha4_p3.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha3_p2plus.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p5.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha3_p5.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p2plus.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p2neg.csv',
 '/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha4_p2plus.csv']

In [3]:
df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p5.csv")
df.head()

,text,sentiment
0,The image shows a group of people standing on ...,1
1,The image shows a long stretch of train tracks...,4
2,This image shows a busy city street with tall ...,2
3,The image shows a street with a small pothole ...,1
4,This image shows a view of clouds from an airp...,4


In [4]:
df.sentiment.value_counts()

sentiment
4    221
0     87
2     75
1     44
3     18
Name: count, dtype: int64

In [5]:
def process_prompt(row):
    return f"{row['text']} - {row['sentiment']}"

def prepare_data(dataframe: pd.DataFrame):
    X_train = []
    X_test = []

    # X_train, X_test = train_test_split(
    #     dataframe,
    #     test_size=0.2,
    #     random_state=42
    # )
    for sentiment in [0, 1, 2, 3, 4]:
        
        train, test = train_test_split(df[df.sentiment == sentiment],
                                       test_size=0.2,
                                       random_state=42)
        X_train.append(train)
        X_test.append(test)

    X_train = pd.concat(X_train)
    X_test = pd.concat(X_test)


    prompt = """What is the sentiment of this description? Please choose an answer from 
        {
            "Positive": 4,
            "SlightlyPositive": 3,
            "Neutral": 2,
            "SlightlyNegative": 1,
            "Negative": 0
        }
    """
    #         {Positive/SlightlyPositive/Neutral/SlightlyNegative/Negative}.
    # sentiment = {
    #     4: "Positive",
    #     3: "SlightlyPositive",
    #     2: "Neutral",
    #     1: "SlightlyNegative",
    #     0: "Negative",
    # }
    # X_train["sentiment"] = X_train["sentiment"].apply(lambda x: sentiment[x])
    # X_test["sentiment"] = X_test["sentiment"].apply(lambda x: sentiment[x])
    # print(len(X_train), len(X_test))
    X_train["input"] = X_train["text"]
    X_test["input"] = X_test["text"]

    X_train["text"] = X_train[["text", "sentiment"]].apply(lambda x: prompt+x["text"]+'='+str(x["sentiment"]), axis=1)
    # X_test["text"] = X_test[["text", "sentiment"]].apply(lambda x: prompt+x["text"]+'='+str(x["sentiment"]), axis=1)
    # X_train["text"] = X_train[["text"]].apply(lambda x: prompt+x+'=', axis=1)
    X_test["text"] = X_test[["text"]].apply(lambda x: prompt+x+'=', axis=1)

    train_data = Dataset.from_pandas(X_train)#.iloc[0:500])
    test_data = Dataset.from_pandas(X_test)

    return train_data, test_data, X_test.drop(["sentiment"], axis=1), X_test["sentiment"]

In [6]:
train_data, test_data, X_test, y_test = prepare_data(df)

In [7]:
train_data

Dataset({
    features: ['text', 'sentiment', 'input', '__index_level_0__'],
    num_rows: 354
})

In [8]:
X_test.head()

,text,input
367,What is the sentiment of this description? Ple...,The image shows a car that has crashed into a ...
8,What is the sentiment of this description? Ple...,The image shows a group of trash cans lined up...
144,What is the sentiment of this description? Ple...,The image shows a view of a densely populated ...
113,What is the sentiment of this description? Ple...,The image shows a car that has been involved i...
61,What is the sentiment of this description? Ple...,The image shows a group of people standing in ...


In [9]:
X_test["text"].iloc[0]

'What is the sentiment of this description? Please choose an answer from \n        {\n            "Positive": 4,\n            "SlightlyPositive": 3,\n            "Neutral": 2,\n            "SlightlyNegative": 1,\n            "Negative": 0\n        }\n    The image shows a car that has crashed into a ditch on the side of a road. The car is upside down, with its wheels in the air and its roof crumpled. There is mud and debris all around the car, and it appears to have sustained significant damage. The image is taken from a low angle, looking up at the car from the bottom of the ditch.='

In [10]:
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig)
import torch

In [30]:
# def init_model(model_name="nvidia/Llama-3.1-Nemotron-70B-Instruct-HF"):
# def init_model(model_name="meta-llama/Llama-3.2-1B"):
# def init_model(model_name="meta-llama/Llama-2-7b-chat-hf"):
def init_model(model_name="nvidia/Llama3-ChatQA-1.5-8B"):
# def init_model(model_name="nvidia/NV-Embed-v2"):
    compute_dtype = getattr(torch, "float16")
    torch.cuda.reset_peak_memory_stats()
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    #     bnb_4bit_compute_dtype=torch.float16,
        llm_int8_enable_fp32_cpu_offload=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True,
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True,
    )
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    return model, tokenizer

In [33]:
model, tokenizer = init_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [13]:
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraModel, LoraConfig

In [30]:
# Define the metric computation function
def compute_metrics(eval_pred):
    f1_metric = evaluate.load("f1", use_auth_token=False)
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"f1": f1["f1"]}

def train(model, tokenizer, train_data, eval_data):
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        # lora_alpha=2,
        # lora_dropout=0.1,
        # r=8,
        bias="none",
        task_type="CAUSAL_LM",
    )

    training_arguments = TrainingArguments(
        output_dir="../logs/log",
        num_train_epochs=100,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=8,
        optim="paged_adamw_32bit",
        save_steps=0,
        logging_steps=25,
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=True,
        bf16=False,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="cosine",
        report_to="tensorboard",
        evaluation_strategy="epoch",
        gradient_checkpointing=True,
        eval_accumulation_steps=2,
    )


    trainer = SFTTrainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=eval_data,
        peft_config=peft_config,
        dataset_text_field="text",
        tokenizer=tokenizer,
        args=training_arguments,
        # compute_metrics=compute_metrics,
        packing=False,
        max_seq_length=1064,
    )

    trainer.train()

    output_dir = "../logs/results/trained_model"

    trainer.save_model(output_dir)

In [31]:
train(model, tokenizer, train_data, test_data)

/home/neemias/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/neemias/anaconda3/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/neemias/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/neemias/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is depr

Map:   0%|          | 0/354 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

/home/neemias/anaconda3/lib/python3.11/site-packages/accelerate/utils/imports.py:313: UserWarning: Intel Extension for PyTorch 2.4 needs to work with PyTorch 2.4.*, but PyTorch 2.5.1 is found. Please switch to the matching version and run again.
  warnings.warn(
You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
/home/neemias/anaconda3/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,1.433744
2,1.645800,0.738938
3,0.961700,0.590182
4,0.609100,0.543468
5,0.513700,0.533871
6,0.464900,0.535290
7,0.412100,0.556756
8,0.367600,0.588780
9,0.327200,0.608576
10,0.283600,0.642707


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

KeyboardInterrupt: 

In [32]:
from transformers import pipeline
from tqdm.notebook import tqdm

def inference(pipe, prompt):
    result = pipe(prompt)
    # print(result)
    # print()
    answer = result[0]['generated_text'].split("=")[-1]
    return answer

def predict(X_test, model, tokenizer):
    y_pred = []
    pipe = pipeline(task="text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens=1,
                    temperature=0.01,
                    do_sample=True
                    # device='cuda'
                    )
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["text"]
        answer = inference(pipe, prompt)
        # print(answer)
        y_pred.append(int(answer))
    return y_pred

In [33]:
y_pred = predict(X_test, model, tokenizer)

  0%|          | 0/91 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/neemias/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [34]:
y_pred[0]

0

In [35]:
sentiment = {
    "Positive": 4,
    "SlightlyPositive": 3,
    "Neutral": 2,
    "SlightlyNegative": 1,
    "Negative": 0,
}

In [36]:
np.unique(y_pred)

array([0, 1, 2, 3, 4])

In [37]:
from sklearn.metrics import classification_report, f1_score

In [56]:
print(f"{f1_score(y_true=[sentiment[v] for v in y_test.to_list()], y_pred=[sentiment[v] for v in y_pred], average='weighted'):.2f}")

0.65


In [1]:
f1_score(y_test, y_pred)

NameError: name 'f1_score' is not defined

In [38]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83        18
           1       0.33      0.44      0.38         9
           2       0.75      0.60      0.67        15
           3       0.00      0.00      0.00         4
           4       0.89      0.93      0.91        45

    accuracy                           0.77        91
   macro avg       0.56      0.56      0.56        91
weighted avg       0.76      0.77      0.76        91

